In [9]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from bs4 import BeautifulSoup
import calendar
import datetime

In [10]:
risk_free_rate_url = "https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield"

In [11]:
path = os.path.join(os.getcwd(),"Data")

def download_risk_free_rate():
    """
    Downloads the Risk Free Rate file.

    risk_free_rate_url = "https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield"

    creates the driver.
    opens the risk_free_rate_url.
    downloads the file.

    Methods:
    --------

    create_driver : creates the chrome driver.

    download : extracts the data from the page and saves to a csv file.

    """
    def create_driver():
        """
        Creates a Chrome Driver.

        Returns
        --------
        driver : driver
            chrome web driver.
        """

        chromeOptions = webdriver.ChromeOptions()
        chromeOptions.add_argument("--headless")
        chromeOptions.add_experimental_option("prefs",{"download.default_directory":path})
        driver = webdriver.Chrome(ChromeDriverManager().install(), options = chromeOptions)
        return driver

    def download():
        """
        downloads the risk free rate file.

        """
        ele = driver.find_element_by_xpath('//*[@id="interestRateTimePeriod"]')
        ele = Select(ele)
        ele.select_by_visible_text("All") 

        btn = driver.find_element_by_xpath('/html/body/form/div[8]/div/div[1]/div/div[2]/div/div/div/div[1]/div[2]/div/table/tbody/tr/td/div/div[3]/div[2]/input')
        btn.click()
        time.sleep(15)
        soup = BeautifulSoup(driver.page_source,'html')
        driver.quit()
        table = soup.find_all("table", {"class": "t-chart"})
        risk_free_rate = pd.read_html(str(table))[0]
        # risk_free_rate.to_csv(os.path.join(path,"inRiskFreeRate.csv"),index=None)
        risk_free = risk_free_rate[["Date","3 mo"]]
        risk_free_rate["Rate"] = risk_free_rate["3 mo"]
        risk_free.columns = ["Date","Rate"]
        risk_free.dropna(inplace=True)
        risk_free.to_csv(os.path.join(path,"RiskFreeRate.csv"),index=None)
    driver = create_driver()
    driver.get(risk_free_rate_url)
    download()

In [5]:
# download_risk_free_rate()